In [1]:
import sys
from pathlib import Path
import numbers
import random
import json
from itertools import chain
print(sys.executable)
sys.path.extend(['..', '../..'])
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, FastICA
from sklearn.ensemble import IsolationForest
from sklearn.mixture import GaussianMixture
from skimage import transform, color
import cv2
import imageio
from quant.experiment.clustering import Clustering
from base.datasets.wsi_reader import WSIReader

/well/rittscher/users/achatrian/.conda/envs/pyenvclone/bin/python


In [3]:
e = Clustering('guassians',
              ('StandardScaler', 'PCA', 'GaussianMixture'),
              (StandardScaler(), PCA(n_components=5), GaussianMixture(n_components=5)),
              caching_path='/well/rittscher/users/achatrian/pipeline_cache')

In [4]:
remove_outliers = False
e.read_data_from_dir('/well/rittscher/projects/prostate-gland-phenotyping/WSI/data/features', max_memory_use=1e10)
if remove_outliers:
    e.remove_outliers()

  5%|▌         | 1/20 [00:07<02:13,  7.02s/it]

Memory usage: 103230105


 10%|█         | 2/20 [00:13<02:02,  6.79s/it]

Memory usage: 167556857


 15%|█▌        | 3/20 [00:18<01:46,  6.26s/it]

...

Memory usage: 981169722


 95%|█████████▌| 19/20 [02:16<00:08,  8.72s/it]

Memory usage: 1077062791


100%|██████████| 20/20 [02:23<00:00,  8.43s/it]


Memory usage: 1100381981
20 feature files were loaded.


In [ ]:
print("Num points ", len(e.x))
print("Num fields ", len(e.x.columns))
print(e.x.index.levels[0])

In [ ]:
e.run()

In [ ]:
e.plot_clusters()#colors=list(str(i * 0.1) for i in range(10)))

In [ ]:
# read in csv file
label_file = '/well/rittscher/projects/prostate-gland-phenotyping/WSI/data/other/TA206_Scores_V2_190718.csv'
labels = pd.read_csv(label_file, sep='\t')
labels = labels.set_index('file_name')

In [ ]:
erg = labels['ERG H Score'].apply(lambda x: int(x) if x.isdigit() else 0)
erg = erg.apply(lambda x: int(x) > 0)
erg = erg.iloc[0::2]  # remove benign samples from spreadseet
erg.index = [s[:-5] for s in erg.index]

In [ ]:
e.x.index.levels[0]

In [ ]:
erg.index

In [ ]:
erg.loc[e.x.index.levels[0][0]]

In [ ]:
# plot feature space against ERG:
colors = ['r', 'b']
for slide, erg_v in erg.iteritems():  # erg is a Series
    color_ = colors[int(erg_v)]
    try:
        slide_x = e.x.loc[slide]
        plt.scatter(slide_x.iloc[:, 0], slide_x.iloc[:, 1], color=color_)
    except KeyError:
        pass

In [ ]:
# PTEN
pten = labels['PTEN H Score RESCORE BETTER STAINED TMA'].apply(lambda x: int(x) if x.isdigit() else 0)
pten = pten.apply(lambda x: int(x) > 0)
pten = pten.iloc[0::2]  # remove benign samples from spreadseet
pten.index = [s[:-5] for s in pten.index]

In [ ]:
# plot feature space against ERG:
colors = ['r', 'b']
for slide, pten_v in pten.iteritems():  # erg is a Series
    color_ = colors[int(pten_v)]
    try:
        slide_x = e.x.loc[slide]
        plt.scatter(slide_x.iloc[:, 0], slide_x.iloc[:, 1], color=color_)
    except KeyError:
        pass

In [ ]:
labels

In [ ]:
# pi
# PTEN
proliferation = labels['Proliferation index Mib-1 0-100% (only invasive tumour scored, not PIN or intraductal Ca or inflammatory cells. In the benign, only luminal ep cells scored)'].apply(lambda x: int(x) if x.isdigit() else 0)
proliferation = proliferation.iloc[0::2]  # remove benign samples from spreadseet
proliferation.index = [s[:-5] for s in proliferation.index]

In [ ]:
# plot feature space against ERG:
colors = ['r', 'b', 'g', 'y']
for slide, pten_v in pten.iteritems():  # erg is a Series
    color_ = colors[int(pten_v)]
    try:
        slide_x = e.x.loc[slide]
        plt.scatter(slide_x.iloc[:, 0], slide_x.iloc[:, 1], color=color_)
    except KeyError:
        pass

In [ ]:
# get gland examples manually ?

In [ ]:
def get_examples(self, data_dir=None, n_examples=5, mpp=0.2):
    r"""This methods assumes that the dataframe's index corresponds to the bounding box of tissue elements
    :param n_examples: how many image examples to extract for image per cluster
    :param mpp:
    """
    if self.x is None or self.y is None:
        raise ValueError("Data has not been read / processed yet for cluster extraction")
    if data_dir is None:
        data_dir = Path(self.loaded_paths[0]).parents[2]
        assert (data_dir/'data').is_dir(), "data_dir should contain dir 'data'"
    self.clusters = np.unique(self.y)
    examples = []
    for i, cluster in enumerate(tqdm_notebook(self.clusters, desc='clusters')):
        x_cluster = self.x.iloc[(self.y == cluster).to_numpy().squeeze()]
        examples.append(dict())
        for subset_id in tqdm_notebook(self.x.index.levels[0], desc='subsets'):
            try:
                subset_path = next((data_dir/(subset_id + sfx)) for sfx in ['.ndpi', '.svs', '.dzi'])
            except StopIteration:
                raise ValueError(f"Data dir does not contain image for {subset_id}")
            examples[i][subset_id] = []
            opt = WSIReader.get_reader_options(include_path=False)
            reader = WSIReader(opt, subset_path)
            try:
                x_subset = x_cluster.loc[subset_id]
            except KeyError:
                continue
            sample = x_subset.sample(n=n_examples, replace=True)
            for bb_s, row in sample.iterrows():
                x, y, w, h = tuple(int(d) for d in bb_s.split('_'))
                image = np.array(reader.read_region((x, y), 0, (w, h)))  # changed level from None to 0 !!!
                if image.shape[2] == 4:  # assume 4 channels images are RGBA
                    image = color.rgba2rgb(image)
                if image.max() <= 1.0 and image.min() >= 0.0:
                    image = image * 255.0
                image = image.astype(np.uint8)
                examples[i][subset_id].append(image)
    return examples

In [ ]:
examples = get_examples(e, n_examples=30)

In [ ]:
def save_examples_grid(self, save_dir, examples, image_size=512):
    r""""""
    save_dir = Path(save_dir)
    n_clusters = len(examples)
    n_subsets = len(examples[0])
    n_examples = len(next(iter(examples[0].values())))
    print(f"n subsets, clusters: {n_subsets}, {n_clusters}")
    for n in tqdm_notebook(range(n_examples)):
        grid = np.zeros((image_size * n_subsets, image_size * n_clusters, 3))
        if n == 0:
            print(f"Grid size: {grid.size}")
        for j, cluster_examples in enumerate(examples):
            for i, (subset_id, subset_examples) in enumerate(cluster_examples.items()):
                if len(subset_examples) >= n+1:
                    example = np.array(subset_examples[n])
                    max_dim = max(example.shape[:2])
                    padded = np.pad(example, (
                            (0, max(0, max_dim - example.shape[0])),
                            (0, max(0, max_dim - example.shape[1])),
                            (0, 0)
                        ), 'constant')
                    resized = transform.resize(padded, output_shape=(image_size,)*2)
                    resized = (resized * 255.0).astype(np.uint8)
                    grid[i*image_size:(i+1)*image_size, j*image_size:(j+1)*image_size] = resized
                    cv2.putText(grid, f'{j}', (j*image_size, i*image_size), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,)*3)
                else:
                    print(f"No examples for cluster {j} in slide {subset_id}")
                    cv2.putText(grid, 'NA', (j*image_size, i*image_size), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,)*3)
        if n_subsets > n_clusters * 2:
            first_half, second_half = grid[:image_size*n_subsets//2], grid[image_size*n_subsets//2:]
            grid = np.concatenate((first_half, second_half), axis=1)
        if n_clusters > n_subsets * 2:
            first_half, second_half = grid[:, :image_size*n_clusters//2], grid[:, image_size*n_clusters//2:]
            grid = np.concatenate((first_half, second_half), axis=1)
        imageio.imwrite(save_dir/f'grid{n}.png', grid)
    with open(save_dir/'details.json', 'w') as details:
        json.dump({
            'experiment_name': self.name,
            'files': list(examples[0].keys()),
        }, details)
    print("Done!")

In [ ]:
save_examples_grid(e, '/well/rittscher/users/achatrian/temp', examples, image_size=128)